In [13]:
import os
import shutil
from mmengine.config import Config
import json

In [8]:
current_dir = os.getcwd()

def listdir(*rel_paths):
    return os.listdir(os.path.join(current_dir, *rel_paths))

def isdir(*rel_paths):
    return os.path.isdir(os.path.join(current_dir, *rel_paths))

def join(*rel_paths):
    return os.path.join(current_dir, *rel_paths)

def isfile(*rel_paths):
    return os.path.isfile(join(*rel_paths))

def rmdir(*rel_paths):
    return shutil.rmtree(join(*rel_paths), ignore_errors=True)

In [9]:
work_dirs = [d for d in os.listdir() if isdir(d)]
print(f'work dirs: {len(work_dirs)}')
log_runs = [(work_dir, d) for work_dir in work_dirs for d in listdir(work_dir) if isdir(work_dir,d)]
print(f'log runs: {len(log_runs)}')

work dirs: 4
log runs: 14


In [10]:
failed = [(work_dir,d) for work_dir,d in log_runs if (not isfile(join(work_dir,d,'vis_data', f'{d}.json')) and not isfile(join(work_dir, d, f'{d}.json')))]
print(f'failed: {len(failed)}')
for work_dir, d in failed:
    rmdir(work_dir, d)

failed: 0


In [11]:
work_dirs = [d for d in os.listdir() if isdir(d)]
print(f'work dirs: {len(work_dirs)}')
log_runs = [(work_dir, d) for work_dir in work_dirs for d in listdir(work_dir) if isdir(work_dir,d)]
print(f'log runs: {len(log_runs)}')
# there are still more runs that may have failed, we can identify them by looking at the contents of the json files (does the second last line iteration count match the max_iters in the config?)


work dirs: 4
log runs: 14


In [21]:
train_runs = [(work_dir, d) for work_dir, d in log_runs if isfile(work_dir, d, 'vis_data', f'{d}.json')]

def load_json(work_dir, d):
    with open(join(work_dir, d, 'vis_data', f'{d}.json')) as f:
        data = f.readlines()
        data = [json.loads(line) for line in data]
        return data

def completed(work_dir, d, data):
    iteration = data[-2].get('iter')
    if iteration is None:
        return False
    config = Config.fromfile(join(work_dir, d, 'vis_data', 'config.py'))
    return iteration == config.train_cfg.max_iters

train_runs = [(work_dir, d, load_json(work_dir, d)) for work_dir, d in train_runs if isfile(work_dir, d, 'vis_data', f'{d}.json')]

successes = [(*train_run, completed(*train_run)) for train_run in train_runs]

print([success[-1] for success in successes])
print([success[:2] for success in successes if not success[-1]])

[False, True, False, True, True, True, True, False, True, True, False]
[('setr_vit-l_pup-noaux_8xb2-160k_ade20k-512x512', '20231119_215811'), ('setr_vit-l_pup-noaux_8xb2-160k_ade20k-512x512', '20231119_210546'), ('setr_vit-l_pup_8xb2-160k_ade20k-512x512', '20231119_215811'), ('setr_vit-l_pup_8xb2-160k_ade20k-512x512', '20231119_210223')]
